# DPEE1085 - Deep Learning
## Parte 1: Word2Vec

A primeira parte do trabalho consiste em treinar um codificador Word2Vec seguindo uma das seguintes propostas:

- Opção 1: Treinar Word2Vec utilizando vocabulário na língua portuguesa, utilizando ferramenta padrão com implementação pronta, tal como gensim.
- Opção 2: Treinar Word2Vec utilizando vocabulário na língua inglesa, mas nesse caso implementando manualmente o algoritmo de treinamento utilizando uma biblioteca tal como TensorFlow.

Em ambos os casos os autores devem demonstrar resultados satisfatórios do treinamento, mostrando como o embedding criado agrupa palavras de categoria semanticamente afim, e mostrando resultados coerentes em operações do tipo rei - homem + mulher = rainha.

## Parte 2: LSTM

A segunda parte do trabalho consiste em implementar uma LSTM seguindo uma das seguintes propostas:

- Opção 1: Treinar LSTM para classificar sentimentos (ou classe de documento, ou outro tipo de classificação similar). Neste caso a rede deve ser treinada utilizando embeddings que se aproveitam do codificador Word2Vec implementado na Parte 1.

- Opção 2: Treinar LSTM para uma tarefa de predição da próxima palavra. Neste caso se treina a LSTM usando como entrada uma sequência de palavras de um texto, onde a rede neural deve prever a próxima palavra. Também neste caso a rede neural deve ser treinada utilizando os embeddings criados na Parte 1.

- Opção 3: Treinar LSTM para outra tarefa de predição, não necessariamente ligada a processamento de linguagem natural. Essa opção deve ter uma proposta elaborada em detalhe e aprovada pelo professor. Apenas propostas aprovadas serão consideradas para avaliação.

In [37]:
import sys
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle as pkl
import random

In [2]:
def pickleLoader(pklFile):
    try:
        while True:
            yield pkl.load(pklFile)
    except EOFError:
        pass

In [3]:
#le os arquivos que serão utilizados no W2V

sentences = []
i = 0
files = ["processed_tweets.pkl","processed_tweets_positives.pkl","processed_tweets_negatives.pkl"]

for h,file in enumerate(files):
    with open("files/"+file, 'rb') as f:
        for text in pickleLoader(f):
            #print("ORIGINAL:", text)
            #print(text.split(" "))
            if(h==0):
                sentences.append(text.split(" "))
            else:
                sentences.append(text[1].split(" "))

sentences.append("o o o o o o") # Para evitar o erro de que 

In [4]:
#treina o W2V
modell = Word2Vec(sentences, min_count=1)
modell.train(sentences, total_examples=len(sentences), epochs=10)

(6130637, 6456010)

In [36]:
#teste
print(modell.wv.most_similar (positive=['brasil']))

print("\n", modell.wv.most_similar(positive=['brasil', 'copa'], negative=['alemanha'], topn=1))

print("\n", modell.wv.most_similar(positive=['brasil', 'copa'], negative=['mexico'], topn=1))

print("\n", modell.wv.most_similar(positive=['brasil', 'copa'], negative=['neymar'], topn=1))

print("\n", modell.wv.similarity('mulher', 'homem'))

print("\n", modell.wv['o'])  # numpy vector of a word

[('mexico', 0.828896164894104), ('portugal', 0.8126705288887024), ('belgica', 0.7991598844528198), ('hexa', 0.7974234223365784), ('franca', 0.7863509058952332), ('flamengo', 0.7760998010635376), ('uruguai', 0.7705225944519043), ('argentina', 0.7584976553916931), ('europa', 0.7546849846839905), ('ssskdks', 0.7510425448417664)]

 [('hexa', 0.6413854956626892)]

 [('bane', 0.6740082502365112)]

 [('penta', 0.6982816457748413)]

 [('fenomeno', 0.8078943490982056)]

 0.7118069339769482

 [-0.11549    -0.27647    -0.00396349  0.12098481 -0.07960594  0.00204572
 -0.16472812 -0.22133563  0.00387899  0.16680218 -0.21729277 -0.09309099
 -0.25117826  0.06916799  0.20069979 -0.17069975  0.0368557  -0.24217759
  0.03492099  0.10246738  0.2730093  -0.35994372  0.01788283  0.2065809
 -0.09640203 -0.08328907 -0.15541989 -0.10900876 -0.2095093  -0.06068301
  0.09834804  0.16888113 -0.02276938 -0.14903912  0.02197512 -0.01991362
  0.04944568  0.4047872  -0.03368277  0.29766044  0.04923915 -0.4209322
  0

In [38]:
#le os arquivos para o LSTM
sentences = []
i = 0
aux = 0
etiqueta = 0.99
files = ["processed_tweets_positives.pkl","processed_tweets_negatives.pkl"]

for h,file in enumerate(files):
    with open("files/"+file, 'rb') as f:
        for text in pickleLoader(f):
                #print("ORIGINAL:", text)
                #print(text.split(" "))
                sent = []
                vec = []
                texto = text
                sent.append(text[1].split(" "))
                
                #preenche com a letra "o" para todos os tweets terem 30 palavras.
                i = 30-len(sent[0])
                for a in range(0,i):
                    sent[0].append("o")
                
                #percorre todas as palavras para pegar o valor do word2vec em float
                v = sent[0]
                for a in range(0,30):
                    vecIn = modell.wv[v[a]]
                    x = []
                    
                    for i in range(len(vecIn)):
                        x.append(float(vecIn[i]))
                    
                    vec.append(x)
                
                sentences.append([h*etiqueta,vec,text])

random.shuffle(sentences)

In [39]:
print(sentences[10])

[0.99, [[-0.41279470920562744, -1.4225856065750122, 0.7126961350440979, -0.8230488300323486, 0.0878543108701706, 1.0251681804656982, -1.4490877389907837, 0.605741560459137, -1.1974543333053589, 1.395904779434204, -0.5721455812454224, -0.3681626617908478, -0.10267823934555054, 0.10268700867891312, 0.7618809342384338, -1.5696901082992554, -0.2117762416601181, -0.05281972512602806, 2.0808265209198, 1.2808237075805664, -1.355541467666626, 1.3377246856689453, 0.5886044502258301, 0.61512690782547, -0.3888024389743805, 3.170107126235962, 0.0536312609910965, -0.6190841197967529, -0.940680205821991, -0.6287885904312134, -0.09852243214845657, -0.8267219662666321, 1.780578851699829, -2.2670938968658447, -0.4049707055091858, -0.6128783822059631, -0.9686928987503052, 1.7423433065414429, 0.48324257135391235, -0.7632449269294739, 1.3614006042480469, 2.4462532997131348, -0.13993124663829803, -1.0013175010681152, 0.365491658449173, 0.5881657004356384, -0.9503272175788879, 1.650057077407837, 1.033866643

In [41]:
from keras.layers import LSTM, Dropout, Dense, Activation, Flatten
from keras.models import Sequential
from keras.optimizers import RMSprop, Adam
from keras.callbacks import ModelCheckpoint
import numpy as np

window_size = 30
word2vec_dim = 100
validation_ratio = 0.02
epochs = 5
batch_size = 256
learning_rate = 0.005

weights_filename = None
if len(sys.argv) >= 2:
    weights_filename = sys.argv[1]

/home/lucas/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [42]:
#prepara os dados para o LSTM

def create_batches(quotes):
    x_batches = []
    y_batches = []
    print('Total of {} quotes found'.format(len(quotes)))
    
    for n, quote in enumerate(quotes):
        if n % 25000 == 0:
            print('...processing quote number {}'.format(n))
        
        x = quote[1]
        y = [quote[0]]
        
        x_batches.append(x) #valores - array do w2c
        y_batches.append(y) #etiquetas - postivo x negativo
    
    return np.array(x_batches), np.array(y_batches)

x_batches, y_batches = create_batches(sentences)

Total of 48373 quotes found
...processing quote number 0
...processing quote number 25000


In [43]:
#define a rede
def RNN():
    model = Sequential()
    
    model.add(LSTM(64, input_shape=(window_size, word2vec_dim),return_sequences=True))
    model.add(Flatten())
    
    model.add(Dense(100, activation='sigmoid'))
    model.add(Activation('relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.add(Activation('relu'))
    
    return model

In [44]:
#instancia a rede
model = RNN()
model.summary()

filepath = "weights-improvement-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1,
                             save_best_only=True, mode='min')
callbacks_list = [checkpoint]

model.compile(loss='mean_squared_error',
              optimizer=Adam(lr=learning_rate), metrics=['categorical_accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 30, 64)            42240     
_________________________________________________________________
flatten_1 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               192100    
_________________________________________________________________
activation_1 (Activation)    (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
_________________________________________________________________
activation_2 (Activation)    (None, 1)                 0         
Total params: 234,441
Trainable params: 234,441
Non-trainable params: 0
_________________________________________________________________


In [45]:
#treina a rede
for epoch in range(epochs):
    print('Training epoch {}'.format(epoch))
    ret = model.fit(x_batches, y_batches, batch_size=batch_size,validation_split=validation_ratio, 
                    callbacks=callbacks_list, shuffle=False)

Training epoch 0
Train on 47405 samples, validate on 968 samples
Epoch 1/1
47405/47405 [==============================] - 31s 648us/step - loss: 0.1931 - categorical_accuracy: 1.0000 - val_loss: 0.1780 - val_categorical_accuracy: 1.0000

Epoch 00001: val_loss improved from inf to 0.17798, saving model to weights-improvement-0.18.hdf5
Training epoch 1
Train on 47405 samples, validate on 968 samples
Epoch 1/1
47405/47405 [==============================] - 23s 489us/step - loss: 0.1782 - categorical_accuracy: 1.0000 - val_loss: 0.1709 - val_categorical_accuracy: 1.0000

Epoch 00001: val_loss improved from 0.17798 to 0.17091, saving model to weights-improvement-0.17.hdf5
Training epoch 2
Train on 47405 samples, validate on 968 samples
Epoch 1/1
47405/47405 [==============================] - 23s 482us/step - loss: 0.1712 - categorical_accuracy: 1.0000 - val_loss: 0.1712 - val_categorical_accuracy: 1.0000

Epoch 00001: val_loss did not improve from 0.17091
Training epoch 3
Train on 47405 sam

In [46]:
#testa a rede
acertos = 0
for valTeste,val in enumerate(x_batches):
    result = model.predict_on_batch(x_batches[(valTeste):(valTeste+1),:,:])
    resultado = 1
    esperado =  1
    
    if(result < 0.5):
        resultado = 0
    elif (result > 1):
        resultado = 2
    
    if(y_batches[(valTeste):(valTeste+1)] < 0.5):
        esperado = 0
    
    if(esperado == resultado):
        acertos = acertos + 1
    
    if(valTeste%10000 == 0):
        resultado = "Negative"
        esperado =  "Negative"
        
        if(result < 0.5):
            resultado = "Positive"
        elif (result > 1):
                resultado = "Error"
        
        if(y_batches[(valTeste):(valTeste+1)] < 0.5):
            esperado = "Positive"
        
        print("\n", sentences[valTeste][2][1])
        print("Resultado = ", resultado, "  Esperado = ",esperado)



 segue galera usando bot legal
Resultado =  Positive   Esperado =  Positive

 liga pra gente pf
Resultado =  Positive   Esperado =  Negative

 gente faz cha mate dai tambem posso fazer tutorial chimarrao segundos mandar
Resultado =  Positive   Esperado =  Positive

 one direction hiatus enquanto cada fazendo projetos solo banda acabou
Resultado =  Positive   Esperado =  Positive

 poooooxa nesse caso ainda vou passar sugestao time produto
Resultado =  Positive   Esperado =  Negative


In [47]:
# % de acertos
erros = (len(y_batches)-acertos)
px = (100/len(y_batches)) * acertos

print("Acertos = ", acertos, "Erros = ", erros)
print(px, "%", " acertos")

Acertos =  36887 Erros =  11486
% 76.25534905835902  acertos
